In [1]:
import pandas as pd 
import numpy as np
import pyblp
from sklearn.linear_model import LinearRegression

In [2]:
product_data = pd.read_csv('../data/market_integrates_2.csv')
print(product_data)

     market_ids  firm_ids  characteristic1  characteristic2     prices  \
0             1         1         5.215031         5.912650  22.025552   
1             1         2         5.580586         5.686295  22.445811   
2             1         3         5.477453         5.063807  24.633280   
3             1         4         5.437327         5.368511  20.551346   
4             1         5         5.341661         5.910085  22.982670   
..          ...       ...              ...              ...        ...   
995         100         6         5.657224         5.098946  22.498758   
996         100         7         5.922187         5.442646  20.409394   
997         100         8         5.273980         5.814937  26.659509   
998         100         9         5.690448         5.726883  23.872070   
999         100        10         5.993946         5.298736  17.807141   

     marginal_cost    shares   profits   markups  e_quantity    capital  \
0        13.315867  0.032526  0.2832

In [20]:
# # Getting the dataset estimation ready with at least a supply_side instrument/ demand_side instrument 
# product_data_complete = product_data.rename(columns={'labor':'supply_instrument0',
#                                                     #   'capital':'supply_instruments1',
#                                                         # 'marginal_cost': 'supply_instrument2'
#                                                         })



In [21]:
# the endog regressor
X1_formulation = pyblp.Formulation('1  + prices+ characteristic1 + characteristic2')

# the only regressor on which we want random coeff 
X2_formulation = pyblp.Formulation('0 +  prices')

# supply side charact 
# X3_formulation = pyblp.Formulation('1 + log(labor) + log(capital)')

# formulation for the cost thing (assume cost fomrulation being labor and capital)
product_formulations = (X1_formulation,
                         X2_formulation
                           #  X3_formulation
                           )
print(product_formulations)

(1 + prices + characteristic1 + characteristic2, prices)


In [22]:
mc_integration = pyblp.Integration('monte_carlo', size=50, specification_options={'seed': 0})


In [23]:
mc_problem = pyblp.Problem(product_formulations, product_data, integration=mc_integration, add_exogenous=True)
print(mc_problem)

Initializing the problem ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          
Dimensions:
 T    N     F    I     K1    K2    MD 
---  ----  ---  ----  ----  ----  ----
100  1000  10   5000   4     1     3  

Formulations:
       Column Indices:           0       1            2                3       
-----------------------------  ------  ------  ---------------  ---------------
 X1: Linear Characteristics      1     prices  characteristic1  characteristic2
X2: Nonlinear Characteristics  prices                                          


In [24]:
bfgs = pyblp.Optimization('bfgs', {'gtol': 1e-4})


In [25]:
results1 = mc_problem.solve(sigma=np.ones((1, 1)), optimization=bfgs)


Solving the problem ...

Nonlinear Coefficient Initial Values:
Sigma:     prices    
------  -------------
prices  +1.000000E+00
Starting optimization ...

GMM   Optimization   Objective   Fixed Point  Contraction  Clipped    Objective      Objective      Gradient                  
Step   Iterations   Evaluations  Iterations   Evaluations  Shares       Value       Improvement       Norm           Theta    
----  ------------  -----------  -----------  -----------  -------  -------------  -------------  -------------  -------------
 1         0             1           608         1924         0     +6.644902E-19                 +1.458146E-08  +1.000000E+00

Optimization completed after 00:00:00.
Computing the Hessian and and updating the weighting matrix ...
Computed results after 00:00:01.

Problem Results Summary:
GMM     Objective      Gradient                    Clipped  Weighting Matrix
Step      Value          Norm          Hessian     Shares   Condition Number
----  -------------